In [1]:
import pandas as pd
import watexr_forecast2 as wx
import matplotlib.pyplot as plt
import warnings
import functools
import datetime as dt
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, Markdown, Image, clear_output

warnings.simplefilter('ignore')

# WATExR: Seasonal forecasts

## Select forecast parameters

In [2]:
def get_forecast_data(download, year, season):
    # Path to historic (EWEMBI) dataset for 1981 - 2010
    ewembi_path = r'../../Data/Meteorological/01_ewembi_obs/ewembi_obs_1981-2010.dat'

    # CSV to create summarising EWEMBI quantiles (optional - see Section 3)
    quant_path = r'../../Data/Meteorological/01_ewembi_obs/ewembi_obs_quantiles_1981-2010.csv'
    
    # S4 path
    s4_path = r'../../Data/Meteorological/05_temporary_forecast_data/Morsa/CLIMATE'

    # Download and bias correct S4 data
    if download:
        res = wx.get_seasonal_forecast(year, season)
        
    # 

    # Summarise EWEMBI data
    wx.calculate_ewembi_quantiles(ewembi_path, 
                                  quant_path, 
                                  quants=[0.05, 0.33, 0.67, 0.95],
                                  names=['date', 'time', 'uas', 'vas', 'ps', 'tas', 'pr', 'hurs', 'petH'],
                                 )

    # Average seasonal data
    s4_df = wx.aggregate_seasonal_forecast(season, 
                                           year,
                                           par_list=['tas', 'pr', 'wind'],
                                           names=['date', 'time', 'uas', 'vas', 'ps', 'tas', 'pr', 'hurs', 'petH'],
                                          )

    # Classify forecast
    res_dict = wx.compare_s4_to_ewembi(quant_path, 
                                       season, 
                                       s4_df, 
                                       par_list=['tas', 'pr', 'wind'],
                                       normal_quants=[0.33, 0.67], 
                                       extreme_quants=[0.05, 0.95],
                                      )

    # Read table of historic performance
    xl_path = r'../../Data/Meteorological/04_tercile_plots/seasonal_forecast_performance_1981-2010.xlsx'
    perf_df = pd.read_excel(xl_path)

    # Make summary image
    forecast_png = r'forecast_output/climate_forecast_summary.png'
    clim_plot = wx.make_climate_forecast_png(season, res_dict, perf_df, forecast_png)

In [3]:
def get_months(season):
    """ Get the start and end months for the specified season.
    """
    if season == 'winter':
        return ['November', 'January']
    elif season == 'spring':
        return ['February', 'April']
    elif season == 'early_summer':
        return ['May', 'July']
    elif season == 'late_summer':
        return ['August', 'October']

In [4]:
def make_forecast(b):
    with output:
        
        clear_output()
        
        dl = download_check.value
        year = year_list.value
        season = season_list.value
        
        print('Getting forecast...')
        
        get_forecast_data(dl, year, season)
        
        # Introduction
        if season == 'winter':
            end_year = year + 1
        else:
            end_year = year

        months = get_months(season)
        date = dt.datetime.today()
        date = date.strftime("%B %d. %Y")

        display(Markdown(f'## Forecast for Lake Vansjø: {months[0]} {year} – {months[1]} {end_year}'))
        display(Markdown(f'### Forecast issued {date}'))

        display(Markdown(('This page shows temperature, rainfall and wind conditions expected '
                          'for south-eastern Norway during the next 3 months. For summer (May-Oct), '
                          'lake water quality forecasts for the western basin of Lake Vansjø are also '
                          'produced, where the aim is to predict ecological status according to the '
                          'Water Framework Directive.')))

        display(Markdown('Weather forecasts are issued four times a year, as follows:'))

        display(Image(r'./forecast_output/forecast_table.png', width=400))

        display(Markdown(('Forecasts are generated using an ensemble of bias-corrected '
                          'seasonal climate forecasts (15 members) provided by the ECMWF System 4. '
                          'Lake ecological status forecasts are based on statistical modelling '
                          '(click [here](https://github.com/icra/WATExR) for further information).')))

        display(Markdown('__________'))

        display(Markdown(f'### Weather forecast for {months[0]} {year} – {months[1]} {end_year}'))

        display(Image(r'./forecast_output/climate_forecast_summary.png', width=800))

        display(Markdown('__________'))

        display(Markdown(f'### Lake water quality forecast for {months[0]} {year} – {months[1]} {end_year}'))

        display(Image(r'./forecast_output/quality_forecast_summary.png', width=800))

In [5]:
# User selection widgets
download_check = widgets.Checkbox(value=False,
                                  description='Update/replace cached seasonal data?',
                                  disabled=False,
                                  style={'description_width':'initial'},
                                 )

year_list = widgets.Dropdown(options=range(1982, 2011),
                             value=2000,
                             description='Select year:',
                             disabled=False,
                            )

season_list = widgets.Dropdown(options=(('Winter (NDJ)', 'winter'), 
                                        ('Spring (FMA)', 'spring'), 
                                        ('Early summer (MJJ)', 'early_summer'), 
                                        ('Late summer (ASO)', 'late_summer'),
                                       ),
                               value='winter',
                               description='Select season:',
                               disabled=False,
                              )

button = widgets.Button(description='Start',
                        layout=Layout(width='200px', height='30px'),
                        style={'font_weight': 'bold'},
                       )

output = widgets.Output()

display(download_check, year_list, season_list, button, output)

button.on_click(make_forecast)

Checkbox(value=False, description='Update/replace cached seasonal data?', style=DescriptionStyle(description_w…

Dropdown(description='Select year:', index=18, options=(1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, …

Dropdown(description='Select season:', options=(('Winter (NDJ)', 'winter'), ('Spring (FMA)', 'spring'), ('Earl…

Button(description='Start', layout=Layout(height='30px', width='200px'), style=ButtonStyle(font_weight='bold')…

Output()